In [ ]:
import time
import os
import pyodbc
from shareplum import Site
from shareplum import Office365
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from shareplum.site import Version
user = os.getenv('USERPROFILE')
pd.set_option('max_columns',None)

In [ ]:
# def split(a, n):
#     k, m = divmod(len(a), n)
#     return (a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))

**Setting Path**

In [ ]:
pwd_path = os.path.join(user,
                        'OneDrive - PHOON HUAT PTE LTD\Documents\Password')
cc_path = os.path.join(user, 'PHOON HUAT PTE LTD\Finance - Sales SAP Export\workfile\Yuhui',
                       r'210203 Credit control approval route')
log_path = os.path.join(user, 'PHOON HUAT PTE LTD\Finance - Sales SAP Export\workfile\Yuhui',
                       r'210203 Credit control approval route\Update Log')

In [ ]:
# Database Details
os.chdir(pwd_path)
database = pd.read_csv('database_pwd.txt', index_col='Source')

# Microsoft SQL Server Connection
sql_str = database.loc['sql']['con_str']
sql_conn = pyodbc.connect(sql_str)
print('Connected to Microsoft SQL Server')

# Office user and pwd
office = pd.read_csv('office.txt')
office_uid = office['username'].item()
office_pwd = office['password'].item()

authcookie = Office365('https://phoonhuatsg.sharepoint.com',
                       username=office_uid, password=office_pwd).GetCookies()
site = Site('https://phoonhuatsg.sharepoint.com/sites/Intranet/Sales',
            authcookie=authcookie)
print('Connected to Sharepoint')

# Read Backup

In [ ]:
os.chdir(cc_path)
cc_back_df = pd.read_csv('cc_backup.csv', dtype={'CUSTNO': str, 'CUSTNAME': str, 'CREDIT_TERM': str,
                                                 'CREDITLIMIT': str, 'PAYDEX_LM': str, 'PAYDEX': str,
                                                 'LM SALES': str, 'LM-1 SALES': str, 'LM-2 SALES': str,
                                                 'LM-3 SALES': str, 'LM-4 SALES': str, 'LM-5 SALES': str,
                                                 'LM AR': str, 'LM-1 AR': str, 'LM-2 AR': str, 'LM-3 AR': str,
                                                 'LM-4 AR': str, 'LM-5 AR': str, '>6 AR': str, '>12 AR': str,
                                                 'group': str, 'G1': str, 'G2': str, 'UEN': str, 'ALL': str})
print(cc_back_df.shape)

# Get data from database

In [ ]:
# Initialize required dates
today = datetime.today()
one_month = today - relativedelta(months=1)
two_month = today - relativedelta(months=2)
three_month = today - relativedelta(months=3)
four_month = today - relativedelta(months=4)
five_month = today - relativedelta(months=5)
six_month = today - relativedelta(months=6)
one_year_ago = today - relativedelta(years=1)

# Sales date
today_str = today.strftime('%Y%m%d')
six_months_str = six_month.strftime('%Y%m%d')

# AR POSTINGDATE MONTH
lm_ar = one_month.strftime('%Y%m')
lm_2_ar = two_month.strftime('%Y%m')
lm_3_ar = three_month.strftime('%Y%m')
lm_4_ar = four_month.strftime('%Y%m')
lm_5_ar = five_month.strftime('%Y%m')
lm_6_ar = six_month.strftime('%Y%m')
lm_12_ar = one_year_ago.strftime('%Y%m')

In [ ]:
kna1_query = """
SELECT kna1.CUSTNO, kna1.CUSTNAME, kna1.TERM AS CREDIT_TERM, kna1.CREDITLIMIT, 
kna1.PAYDEX_LM, kna1.PAYDEX, kna1.UEN, sp.[TO] AS EMAIL
FROM 
test.dbo.kna1
LEFT JOIN
test.dbo.spemail AS sp
ON kna1.SALESPERSON = sp.SP
"""

sales_query = """
SELECT 'LM SALES' AS MONTH, sales.CUSTNO, SUM(sales.NETVALUE) AS L6M_SALES
FROM
(SELECT * FROM test.dbo.sales_cost UNION ALL 
SELECT * FROM test.dbo.sales_cost_current UNION ALL 
SELECT * FROM test.dbo.sales_cost_lm) sales
WHERE SUBSTRING(sales.BILLDATE,1,6) = {}
GROUP BY sales.CUSTNO
UNION ALL
SELECT 'LM-1 SALES' AS MONTH, sales.CUSTNO, SUM(sales.NETVALUE) AS L6M_SALES
FROM
(SELECT * FROM test.dbo.sales_cost UNION ALL 
SELECT * FROM test.dbo.sales_cost_current UNION ALL 
SELECT * FROM test.dbo.sales_cost_lm) sales
WHERE SUBSTRING(sales.BILLDATE,1,6) = {}
GROUP BY sales.CUSTNO
UNION ALL
SELECT 'LM-2 SALES' AS MONTH, sales.CUSTNO, SUM(sales.NETVALUE) AS L6M_SALES
FROM
(SELECT * FROM test.dbo.sales_cost UNION ALL 
SELECT * FROM test.dbo.sales_cost_current UNION ALL 
SELECT * FROM test.dbo.sales_cost_lm) sales
WHERE SUBSTRING(sales.BILLDATE,1,6) = {}
GROUP BY sales.CUSTNO
UNION ALL
SELECT 'LM-3 SALES' AS MONTH, sales.CUSTNO, SUM(sales.NETVALUE) AS L6M_SALES
FROM
(SELECT * FROM test.dbo.sales_cost UNION ALL 
SELECT * FROM test.dbo.sales_cost_current UNION ALL 
SELECT * FROM test.dbo.sales_cost_lm) sales
WHERE SUBSTRING(sales.BILLDATE,1,6) = {}
GROUP BY sales.CUSTNO
UNION ALL
SELECT 'LM-4 SALES' AS MONTH, sales.CUSTNO, SUM(sales.NETVALUE) AS L6M_SALES
FROM
(SELECT * FROM test.dbo.sales_cost UNION ALL 
SELECT * FROM test.dbo.sales_cost_current UNION ALL 
SELECT * FROM test.dbo.sales_cost_lm) sales
WHERE SUBSTRING(sales.BILLDATE,1,6) = {}
GROUP BY sales.CUSTNO
UNION ALL
SELECT 'LM-5 SALES' AS MONTH, sales.CUSTNO, SUM(sales.NETVALUE) AS L6M_SALES
FROM
(SELECT * FROM test.dbo.sales_cost UNION ALL 
SELECT * FROM test.dbo.sales_cost_current UNION ALL 
SELECT * FROM test.dbo.sales_cost_lm) sales
WHERE SUBSTRING(sales.BILLDATE,1,6) = {}
GROUP BY sales.CUSTNO
""".format(lm_ar, lm_2_ar, lm_3_ar, lm_4_ar, lm_5_ar, lm_6_ar)

ar_query = """
SELECT 'LM AR' AS MONTH, CUSTNO, SUM(AMOUNT) AS AR
FROM test.dbo.aging
WHERE SUBSTRING(POSTINGDATE,1,6) = {}
GROUP BY SUBSTRING(POSTINGDATE,1,6), CUSTNO
UNION ALL
SELECT 'LM-1 AR' AS MONTH, CUSTNO, SUM(AMOUNT) AS AR
FROM test.dbo.aging
WHERE SUBSTRING(POSTINGDATE,1,6) = {}
GROUP BY SUBSTRING(POSTINGDATE,1,6), CUSTNO
UNION ALL
SELECT 'LM-2 AR' AS MONTH, CUSTNO, SUM(AMOUNT) AS AR
FROM test.dbo.aging
WHERE SUBSTRING(POSTINGDATE,1,6) = {}
GROUP BY SUBSTRING(POSTINGDATE,1,6), CUSTNO
UNION ALL
SELECT 'LM-3 AR' AS MONTH, CUSTNO, SUM(AMOUNT) AS AR
FROM test.dbo.aging
WHERE SUBSTRING(POSTINGDATE,1,6) = {}
GROUP BY SUBSTRING(POSTINGDATE,1,6), CUSTNO
UNION ALL
SELECT 'LM-4 AR' AS MONTH, CUSTNO, SUM(AMOUNT) AS AR
FROM test.dbo.aging
WHERE SUBSTRING(POSTINGDATE,1,6) = {}
GROUP BY SUBSTRING(POSTINGDATE,1,6), CUSTNO
UNION ALL
SELECT 'LM-5 AR' AS MONTH, CUSTNO, SUM(AMOUNT) AS AR
FROM test.dbo.aging
WHERE SUBSTRING(POSTINGDATE,1,6) = {}
GROUP BY SUBSTRING(POSTINGDATE,1,6), CUSTNO
UNION ALL
SELECT '>6 AR' AS MONTH, CUSTNO, SUM(AMOUNT) AS AR
FROM test.dbo.aging
WHERE SUBSTRING(POSTINGDATE,1,6) >= {} AND SUBSTRING(POSTINGDATE,1,6) < {}
GROUP BY SUBSTRING(POSTINGDATE,1,6), CUSTNO
UNION ALL
SELECT '>12 AR' AS MONTH, CUSTNO, SUM(AMOUNT) AS AR
FROM test.dbo.aging
WHERE SUBSTRING(POSTINGDATE,1,6) < {}
GROUP BY SUBSTRING(POSTINGDATE,1,6), CUSTNO
""".format(lm_ar, lm_2_ar, lm_3_ar, lm_4_ar, lm_5_ar,
           lm_6_ar, lm_12_ar, lm_6_ar, lm_12_ar)

cust_g_query = """
SELECT CUSTNO,[group],G1,G2
FROM test.dbo.CUSTGROUP
"""

In [ ]:
kna1_data = pd.read_sql(kna1_query, sql_conn)
sales_data = pd.read_sql(sales_query, sql_conn)
ar_data = pd.read_sql(ar_query, sql_conn)
cust_data = pd.read_sql(cust_g_query, sql_conn)

In [ ]:
sales_pivot = sales_data.pivot_table(
    index='CUSTNO', columns='MONTH', values='L6M_SALES', fill_value=0).reset_index()
sales_pivot.head()

In [ ]:
kna1_sales = kna1_data.merge(sales_pivot, on='CUSTNO', how='left')
print(kna1_sales.shape)
kna1_sales.head()

In [ ]:
kna1_sales_cust = kna1_sales.merge(cust_data, on='CUSTNO', how='left')
kna1_sales_cust.head()

In [ ]:
ar_pivot = ar_data.pivot_table(
    index='CUSTNO', columns='MONTH', values='AR').reset_index()
ar_pivot.head()

In [ ]:
df = kna1_sales_cust.merge(ar_pivot, on='CUSTNO', how='left')
# Fill Missing Values
df = df.fillna('')
df = df[['CUSTNO', 'CUSTNAME', 'CREDIT_TERM', 'CREDITLIMIT', 'PAYDEX_LM', 'PAYDEX',
         'LM SALES', 'LM-1 SALES', 'LM-2 SALES', 'LM-3 SALES', 'LM-4 SALES', 'LM-5 SALES',
         'LM AR', 'LM-1 AR', 'LM-2 AR', 'LM-3 AR', 'LM-4 AR', 'LM-5 AR', '>6 AR', '>12 AR',
         'group', 'G1', 'G2', 'UEN', 'EMAIL']]
# lower case email
df['EMAIL'] = df['EMAIL'].str.lower()
print(df.shape)
df.tail()

In [ ]:
cols = ['CUSTNO', 'CUSTNAME', 'CREDIT_TERM', 'CREDITLIMIT', 'PAYDEX_LM', 'PAYDEX',
        'LM SALES', 'LM-1 SALES', 'LM-2 SALES', 'LM-3 SALES', 'LM-4 SALES', 'LM-5 SALES',
        'LM AR', 'LM-1 AR', 'LM-2 AR', 'LM-3 AR', 'LM-4 AR', 'LM-5 AR', '>6 AR', '>12 AR',
        'group', 'G1', 'G2', 'UEN']
df['ALL'] = df[cols].apply(
    lambda row: '_'.join(row.values.astype(str)), axis=1)

# Compare backup with database

## Get new customers and update sharepoint

### Credit Control Finance

In [ ]:
new_customers_df = df[~df['CUSTNO'].isin(cc_back_df['CUSTNO'])]
new_customers_df.shape

In [ ]:
new_cust_upload = new_customers_df[['CUSTNO', 'CUSTNAME', 'CREDIT_TERM', 'CREDITLIMIT', 'PAYDEX_LM',
                                    'PAYDEX', 'LM SALES', 'LM-1 SALES', 'LM-2 SALES',
                                    'LM-3 SALES', 'LM-4 SALES', 'LM-5 SALES', 'LM AR',
                                    'LM-1 AR', 'LM-2 AR', 'LM-3 AR', 'LM-4 AR', 'LM-5 AR',
                                    '>6 AR', '>12 AR', 'group', 'G1', 'G2', 'UEN']]
new_cust_upload_dict = new_cust_upload.to_dict(orient='records')

In [ ]:
# Select Credit Control Finance List and upload
cc_f_list = site.List('Credit Control Finance')
cc_f_list.UpdateListItems(data=new_cust_upload_dict, kind='New')
print('New customers upload success')

### Credit Control (Salesperson)

In [ ]:
new_cust_upload = new_customers_df[[
    'CUSTNO', 'CUSTNAME', 'CREDIT_TERM', 'CREDITLIMIT','EMAIL']]
new_cust_upload_dict = new_cust_upload.to_dict(orient='records')

In [ ]:
# Select Credit Control List and upload
cc_list = site.List('Credit Control')
cc_list.UpdateListItems(data=new_cust_upload_dict, kind='New')
print('New customers upload success')

## Update data that have changes

### Credit Control Finance

In [ ]:
df_update = df[~df['ALL'].isin(cc_back_df['ALL'])]
df_update = df_update[~df_update['CUSTNO'].isin(new_customers_df['CUSTNO'])]

In [ ]:
print(df_update.shape)
df_update.head()

In [ ]:
customer = df_update['CUSTNO'].to_list()

In [ ]:
id_finance_list = []
for cust in customer:
    query = {'Where': [('Eq', 'CUSTNO', cust)]}
    getid = cc_f_list.GetListItems(query=query, fields=['ID'])[0]['ID']
    id_finance_list.append(getid)
    time.sleep(0.1)

In [ ]:
df_update['ID'] = id_finance_list

**Rearrange columns for Credit Control Finance**

In [ ]:
df_update_finance = df_update[['ID', 'CUSTNO', 'CUSTNAME', 'CREDIT_TERM', 'CREDITLIMIT', 'PAYDEX_LM',
                               'PAYDEX', 'LM SALES', 'LM-1 SALES', 'LM-2 SALES',
                               'LM-3 SALES', 'LM-4 SALES', 'LM-5 SALES', 'LM AR',
                               'LM-1 AR', 'LM-2 AR', 'LM-3 AR', 'LM-4 AR', 'LM-5 AR',
                               '>6 AR', '>12 AR', 'group', 'G1', 'G2', 'UEN']]

In [ ]:
df_split_finance = np.array_split(df_update_finance, 10)

In [ ]:
for i in range(10):
    finance_records = df_split_finance[i].to_dict(orient='records')
    cc_f_list.UpdateListItems(data=finance_records, kind='Update')
    print(i)
    time.sleep(5)

### Credit Control (Salesperson)

**New**

In [ ]:
df_cc_sales = df[['CUSTNO', 'CUSTNAME', 'CREDIT_TERM', 'CREDITLIMIT','EMAIL']].copy()

In [ ]:
df_cc_cols = ['CUSTNO', 'CUSTNAME', 'CREDIT_TERM', 'CREDITLIMIT','EMAIL']
df_cc_sales['ALL'] = df_cc_sales[df_cc_cols].apply(
    lambda row: '_'.join(row.values.astype(str)), axis=1)
df_cc_sales.head()

**Old**

In [ ]:
cc_back_df = cc_back_df.fillna('')
df_cc_back = cc_back_df[['CUSTNO', 'CUSTNAME',
                         'CREDIT_TERM', 'CREDITLIMIT','EMAIL']].copy()

In [ ]:
df_cc_b_cols = ['CUSTNO', 'CUSTNAME', 'CREDIT_TERM', 'CREDITLIMIT','EMAIL']
df_cc_back['ALL'] = df_cc_back[df_cc_b_cols].apply(
    lambda row: '_'.join(row.values.astype(str)), axis=1)
df_cc_back.head()

In [ ]:
df_sales_update = df_cc_sales[~df_cc_sales['ALL'].isin(df_cc_back['ALL'])]
df_sales_update = df_sales_update[~df_sales_update['CUSTNO'].isin(
    new_customers_df['CUSTNO'])]

**UPDATE**

In [ ]:
cc_cust = df_sales_update['CUSTNO'].to_list()

In [ ]:
id_cc_list = []
for cust in cc_cust:
    query = {'Where': [('Eq', 'CUSTNO', cust)]}
    getid = cc_list.GetListItems(query=query, fields=['ID'])[0]['ID']
    id_cc_list.append(getid)
    time.sleep(0.1)

In [ ]:
df_sales_update['ID'] = id_cc_list
df_sales_update.head()

In [ ]:
df_update_cc = df_sales_update[['ID', 'CUSTNO',
                                'CUSTNAME', 'CREDIT_TERM', 'CREDITLIMIT','EMAIL']]

In [ ]:
cc_records = df_update_cc.to_dict(orient='records')
cc_list.UpdateListItems(data=cc_records, kind='Update')

In [ ]:
os.chdir(cc_path)

# Pump out as backup

In [ ]:
df.to_csv('cc_backup.csv', index=False)
df_update_cc.to_csv('cc_updated_' + today_str + '.csv', index=False)
df_update_finance.to_csv('finance_updated_' + today_str + '.csv', index=False)
new_customers_df.to_csv('new_cust_' + today_str + '.csv', index=False)

# Add log

In [ ]:
os.chdir(log_path)
log_file = open('update_log.txt',mode ='a')
log_file.write(datetime.now().ctime() + '\n')
log_file.close()
print(datetime.now())

# SharePoint

**Adding List**

In [ ]:
# site.AddList('Credit Control', description='Credit Control for SA/SP',
#              template_id='Custom List')

# site.AddList('Credit Control Finance', description='Credit Control for SA/SP',
#              template_id='Custom List')

# site.AddList('CC Update', description='Credit Control for SA/SP',
#              template_id='Custom List')

**First Upload**<br>Salesperson

In [ ]:
# first_cc_upload = df[[
#     'CUSTNO', 'CUSTNAME', 'CREDIT_TERM', 'CREDITLIMIT','EMAIL']]
# first_cc_upload_dict = first_cc_upload.to_dict(orient='records')
# cc_list = site.List('Credit Control')
# cc_list.UpdateListItems(data=first_cc_upload_dict, kind='New')

Finance

In [ ]:
# cc_f_list = site.List('Credit Control Finance')
# first_cc_f_upload_dict = df.to_dict(orient='records')
# cc_f_list.UpdateListItems(data=first_cc_f_upload_dict, kind='New')

**Huge Query Won't Work**

In [ ]:
# cc_list.GetListItems()

**Deleting List (MUST GET ID for Each query of INDEX)**

In [ ]:
# for index in df['INDEX'].unique():

#     query = {'Where': [('Eq', 'INDEX', str(index))]}

#     cc_item = cc_list.GetListItems(query=query)
#     cc_item_id = [item['ID'] for item in cc_item]

#     if len(cc_item) != 0:
#         cc_list.UpdateListItems(cc_item_id, kind='Delete')
#     time.sleep(120)